In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Preprocess Data

In [1]:
import pandas as pd
from transformers import BertTokenizerFast

# Bước 1: Tải và định dạng dữ liệu
# Hàm load_data đọc các câu và nhãn từ các tệp trong định dạng NER cụ thể:
# Mỗi dòng chứa một từ và thẻ tương ứng.
# Các câu được phân tách bằng các dòng trống.
# Khi gặp dòng trống, câu hiện tại và các nhãn của nó sẽ được thêm vào các danh sách sentences và labels.
def load_data(filepath):
    sentences, labels = [], []
    with open(filepath, 'r') as f:
        sentence, label = [], []
        for line in f:
            if line.startswith('-DOCSTART-') or line == "\n":
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                sentence, label = [], []
            else:
                word, _, _, tag = line.strip().split()
                sentence.append(word)
                label.append(tag)
        if sentence:
            sentences.append(sentence)
            labels.append(label)
    return sentences, labels

# Sau khi tải:
# Các biến sau giữ các danh sách câu và nhãn NER tương ứng.
train_sentences, train_labels = load_data('/content/drive/MyDrive/Colab_Notebooks/MIDTERM_NLP/train.txt') #train.txt
test_sentences, test_labels = load_data('/content/drive/MyDrive/Colab_Notebooks/MIDTERM_NLP/test.txt')  # test.txt
valid_sentences, valid_labels = load_data('/content/drive/MyDrive/Colab_Notebooks/MIDTERM_NLP/valid.txt') #valid.txt

# Khởi tạo các tokenizer:
# BertTokenizerFast được sử dụng để token hóa các câu.
# is_split_into_words=True cho biết rằng các câu đã được chia thành các từ riêng lẻ. => Rất hữu ích khi đã có danh sách các từ và muốn token hóa chúng.
# return_offsets_mapping=True trả về vị trí ký tự bắt đầu và kết thúc của mỗi token trong câu => Giúp căn chỉnh các token với các nhãn tương ứng.
# padding=True đảm bảo rằng tất cả các câu đều có cùng độ dài bằng cách thêm các token đệm (padding tokens) vào cuối các câu ngắn hơn.
# truncation=True cắt bớt các câu dài hơn độ dài tối đa cho phép, đảm bảo rằng tất cả các câu đều có độ dài phù hợp.
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_sentences, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
test_encodings = tokenizer(test_sentences, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
valid_encodings = tokenizer(valid_sentences, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Step 2: Define the Model

In [2]:
from transformers import BertForTokenClassification
import torch

# Tạo một danh sách các thẻ có thể có (như O, B-ORG, ...) và ánh xạ từng thẻ vào một số nguyên duy nhất.

# label_list chứa tất cả các thẻ NER có thể có.
label_list = ["O", "B-ORG", "I-ORG", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]
# label_map là một từ điển ánh xạ mỗi thẻ vào một ID số nguyên,
# giúp dễ dàng chuyển đổi nhãn thành ID để model huấn luyện.
label_map = {label: i for i, label in enumerate(label_list)}

# Hàm encode_labels căn chỉnh các nhãn với các token:
def encode_labels(labels, encodings):
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        #Các token không có nhãn cụ thể sẽ được gán nhãn "O" (đối với các thực thể ngoài).
        doc_enc_labels = [label_map["O"]] * len(doc_offset)
        # Với mỗi token trong một câu, nó gán một ID nhãn tương ứng từ label_map.
        for i, label in enumerate(doc_labels):
            doc_enc_labels[i] = label_map[label]
        encoded_labels.append(doc_enc_labels)
    return encoded_labels
train_labels = encode_labels(train_labels, train_encodings)
test_labels = encode_labels(test_labels, test_encodings)
valid_labels = encode_labels(valid_labels, valid_encodings)

# Mô hình BertForTokenClassification từ Hugging Face được khởi tạo với
# số lượng nhãn đầu ra là kích thước của danh sách các ID tương ứng vs các thẻ NER.
# Mô hình BERT được thiết lập để phân loại token với 9 nhãn đầu ra (một cho mỗi thẻ trong label_list).
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Step 3: Training

In [3]:
from transformers import Trainer, TrainingArguments

# Prepare data for training
# Lớp NERDataset bao bọc các đầu vào đã được token hóa và các nhãn đã được mã hóa để API Trainer có thể sử dụng nó
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # trả về đầu vào token hóa và các nhãn tại một chỉ mục nhất định.
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # trả về số lượng mục trong tập dữ liệu
    def __len__(self):
        return len(self.labels)

# Các tập dữ liệu sau được tạo cho train, test và valid
train_dataset = NERDataset(train_encodings, train_labels)
test_dataset = NERDataset(test_encodings, test_labels)
valid_dataset = NERDataset(valid_encodings, valid_labels)

# Define training arguments
# Các tham số huấn luyện được định nghĩa bằng TrainingArguments:
training_args = TrainingArguments(
    # Thư mục lưu trữ kết quả huấn luyện
    output_dir='./results',
    # evaluation_strategy="epoch" chỉ định việc đánh giá vào cuối mỗi epoch
    #(một lần duyệt qua toàn bộ dữ liệu huấn luyện).
    evaluation_strategy="epoch",
    # learning_rate=2e-5: Tốc độ học của mô hình, xác định mức độ điều chỉnh trọng số sau mỗi lần cập nhật.
    learning_rate=2e-5,
    # Kích thước batch cho mỗi thiết bị trong quá trình huấn luyện.
    per_device_train_batch_size=16,
    # Kích thước batch cho mỗi thiết bị trong quá trình đánh giá.
    per_device_eval_batch_size=16,
    # Số lượng epoch để huấn luyện mô hình.
    num_train_epochs=3,
    # Hệ số giảm trọng số, giúp tránh overfitting bằng cách giảm giá trị của các trọng số lớn.
    weight_decay=0.01,
)

# Define Trainer
# Khởi tạo một đối tượng Trainer từ thư viện Transformers của Hugging Face
#Trainer một lớp giúp bạn dễ dàng huấn luyện và đánh giá mô hình.
trainer = Trainer(
    # model=model: Đây là mô hình bạn muốn huấn luyện.
    #Nó có thể là bất kỳ mô hình nào từ thư viện Transformers, chẳng hạn như BERT, GPT-2, v.v.
    model=model,
    # Đây là các tham số huấn luyện mà đã được định nghĩa ở trên,
    #chẳng hạn như tốc độ học, số lượng epoch, kích thước batch, v.v.
    args=training_args,
    # train_dataset=train_dataset là tập dữ liệu huấn luyện dùng để huấn luyện mô hình.
    train_dataset=train_dataset,
    # eval_dataset=test_dataset là tập dữ liệu đánh giá dùng để đánh giá mô hình trong quá trình huấn luyện.
    eval_dataset=test_dataset
)

# Sử dụng method train() của trainer để train model.
# Sau khi train xong, model sẽ sẵn sàng để đánh giá và sử dụng cho các tác vụ dự đoán.
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.077400,0.030720
2,0.021900,0.026646
3,0.015400,0.023995


TrainOutput(global_step=2634, training_loss=0.03145351578271471, metrics={'train_runtime': 1430.0282, 'train_samples_per_second': 29.456, 'train_steps_per_second': 1.842, 'total_flos': 3525775734542472.0, 'train_loss': 0.03145351578271471, 'epoch': 3.0})

# Step 4: Evaluation

In [6]:
import numpy as np

# Evaluate the model on the validation dataset
# Mô hình được đánh giá trên tập valid_dataset bằng cách sử dụng trainer.evaluate().
#Nó trả về các chỉ số như độ chính xác và độ mất mát trên valid_dataset.
evaluation_results = trainer.evaluate(eval_dataset=valid_dataset)
# in ra {'eval_loss': 0.021987076848745346,
#'eval_runtime': 25.482,
#'eval_samples_per_second': 127.541,
#'eval_steps_per_second': 8.006,
#'epoch': 3.0}
print(evaluation_results)

# Get predictions on the test set
# method trainer.predict() được sử dụng để lấy các dự đoán trên tập test_dataset
#labels là ID nhãn thực từ tập test_dataset.
predictions, labels, _ = trainer.predict(test_dataset)
# predictions chứa các điểm số thô cho mỗi lớp,
#được chuyển đổi thành ID nhãn bằng cách lấy argmax.
predictions = np.argmax(predictions, axis=2)

# Define a function to convert the label IDs back to label names
def convert_labels(predictions, labels, label_list):
    pred_list = []
    true_list = []
    for pred, label in zip(predictions, labels):
        pred_list.append([label_list[p] for p in pred])
        true_list.append([label_list[l] for l in label])
    return pred_list, true_list

# Convert the predictions and labels back to their original format
pred_labels, true_labels = convert_labels(predictions, test_labels, label_list)
# Generate a classification report
from sklearn.metrics import classification_report
# Flatten the lists for easier comparison
true_labels_flat = [item for sublist in true_labels for item in sublist]
pred_labels_flat = [item for sublist in pred_labels for item in sublist]
# Generate and print the classification report
print(classification_report(true_labels_flat, pred_labels_flat, target_names=label_list))
# Ở cột precision với label_name là B-ORG là 0.64,
#nghĩa là 64% trong số các dự đoán B-ORG của mô hình là chính xác.
# Ở cột recall với label_name là B-ORG là 0.57,
#nghĩa là mô hình chỉ nhận diện đúng 57% trong số các mẫu thực sự là B-ORG.
# f1-score: Trung bình điều hòa giữa độ chính xác và độ thu hồi, dùng để cân bằng hai chỉ số này.
# Ở cột f1-score với label_name là B-ORG là 0.60,
#cho thấy mô hình có hiệu suất ở mức trung bình và dự đoán đầu ra không được chính xác lắm
# support: Số lượng mẫu thực sự thuộc label_name đó trong tập test_dataset.
#Ví dụ: B-ORG có 702 mẫu.
# Tổng kết (3 dòng cuối):
# accuracy: Độ chính xác tổng thể của mô hình trên tập kiểm tra là 0.99,
#tức là mô hình dự đoán đúng cho 99% các mẫu.
# macro avg: Trung bình các chỉ số trên cho tất cả các label mà không cân nhắc đến tần suất của label.
#Độ chính xác trung bình là 0.73, độ thu hồi trung bình là 0.70, và f1-score trung bình là 0.71.
# weighted avg: Trung bình các chỉ số nhưng có cân nhắc tần suất của mỗi label.
#Độ chính xác và f1-score ở đây rất cao (0.99) vì các label như I-MISC chiếm đa số.

{'eval_loss': 0.021987076848745346, 'eval_runtime': 24.9254, 'eval_samples_per_second': 130.389, 'eval_steps_per_second': 8.184, 'epoch': 3.0}
              precision    recall  f1-score   support

           O       0.78      0.72      0.75      1668
       B-ORG       0.64      0.57      0.60       702
       I-ORG       0.79      0.71      0.75      1661
       B-PER       0.79      0.69      0.74      1617
       I-PER       0.66      0.70      0.68       257
       B-LOC       0.47      0.50      0.48       216
       I-LOC       0.71      0.72      0.72       835
      B-MISC       0.71      0.72      0.71      1156
      I-MISC       1.00      1.00      1.00    475308

    accuracy                           0.99    483420
   macro avg       0.73      0.70      0.71    483420
weighted avg       0.99      0.99      0.99    483420

